In [1]:
import requests
import json
import pandas as pd
import numpy as np
import re

### Get a user's playlists

In [2]:
def get_plst_ID(user_id, token):
    tidy = lambda s: s[17:]
    base_url = "https://api.spotify.com/v1/users/"
    query = f'{base_url}{user_id}/playlists'

    response = requests.get(query, 
                   headers={"Accept": "application/json",
                            "Content-Type":"application/json", 
                            "Authorization":f"Bearer {token}"})
    json_response = response.json()
    return [tidy(x['uri'])for x in json_response['items']]

### Get a Playlist's Items

In [3]:
def get_ID(user_id, token, playlist_id, market = 'US'):
    tidy = lambda s: s[14:]
    base_url = "https://api.spotify.com/v1/playlists/"
    query = f'{base_url}{playlist_id}/tracks?market={market}'

    response = requests.get(query, 
                   headers={"Accept": "application/json",
                            "Content-Type":"application/json", 
                            "Authorization":f"Bearer {token}"})
    json_response = response.json()
    return [tidy(x['track']['uri'])for x in json_response['items']], [json_response['items'][i]['track']['duration_ms'] for i in range(len(json_response['items']))]

### Get information about a song

In [4]:
def get_song(user_id, token, song_id, market = 'US'):
    
    ## get audio features of the song/track
    song_url = "https://api.spotify.com/v1/audio-features/"
    query = f'{song_url}{song_id}'
    response = requests.get(query, 
                   headers={"Content-Type":"application/json", 
                            "Authorization":f"Bearer {token}"})
    json_response = response.json()
    df_response = pd.json_normalize(json_response)
    
    ## get album information and the first artist
    base_url = "https://api.spotify.com/v1/tracks/"
    query = f'{base_url}{song_id}?market={market}'
    response = requests.get(query, 
                   headers={"Accept": "application/json",
                            "Content-Type":"application/json", 
                            "Authorization":f"Bearer {token}"})
    json_response = response.json()
    album_id = json_response['album']['id']
    artist_id = json_response['artists'][0]['id']
    
    ## get genre of the album
    base_url = "https://api.spotify.com/v1/albums/"
    query = f'{base_url}{album_id}?market={market}'
    response = requests.get(query, 
                   headers={"Accept": "application/json",
                            "Content-Type":"application/json", 
                            "Authorization":f"Bearer {token}"})
    json_response = response.json()
    genre = json_response['genres']
    
    
    df_response['artist_id'] = artist_id if len(artist_id) >0 else None
    df_response['albuma_id'] = album_id if len(album_id) >0 else None
    df_response['genre'] = genre if len(genre) >0 else None
    
    return df_response.drop(['type', 'uri', 'track_href', 'analysis_url'], axis=1).set_index('id')

### Get Recommendations from Spotify API

In [ ]:
### should add a new fuction to check the range of all metrics, like [0,1], [0,100], [-, 0], [0, +], etc.

In [5]:
def recommend(song, limit, time_mt, percent, target_duration_ms, **kwargs):
    seed_tracks = song.index[0]

    min_duration_ms = max(target_duration_ms - time_mt * 1000, 0)
    max_duration_ms = target_duration_ms + time_mt * 1000 
    
    target_energy = song['energy'][0]  ## find the most similar one to the target
    min_energy = max(target_energy * (1 - percent), 0)
    max_energy = target_energy * (1 + percent)
    
    target_instrumentalness = song['instrumentalness'][0]  ## similarly to energy
    min_instrumentalness = max(target_instrumentalness * (1 - percent), 0)
    #max_instrumentalness = target_instrumentalness * (1 + percent)
    
    target_tempo = song['tempo'][0]  ## similarly to energy
    min_tempo = max(target_tempo * (1 - percent), 0)
    max_tempo = target_tempo * (1 + percent)
    target_key = song['key'][0]
    target_danceability=song['danceability'][0]
    target_mode=song['mode'][0]
    
    endpoint_url = "https://api.spotify.com/v1/recommendations?"
    query = f'{endpoint_url}limit={limit}&seed_tracks={seed_tracks}'
    query += f'&target_duration_ms={target_duration_ms}&min_duration_ms={min_duration_ms}&max_duration_ms={max_duration_ms}'
    query += f'&target_energy={target_energy}&min_energy={min_energy}&max_energy={max_energy}'
    query += f'&target_instrumentalness={target_instrumentalness}&min_instrumentalness={min_instrumentalness}'
    query += f'&target_tempo={target_tempo}&min_tempo={min_tempo}&max_tempo={max_tempo}'
    query += f'&target_key={target_key}'
    query += f'&target_danceability={target_danceability}'
    query += f'&target_mode={target_mode}'
    
    if song['genre'][0] is not None:
        seed_genres = song['genre'][0]
        query += f'&seed_genres={seed_genres}'
    if song['artist_id'][0] is not None:
        seed_artist = song['artist_id'][0]
        query += f'&seed_artist={seed_artist}'
    
    if len(kwargs.keys()) > 0:
        query += '&'
        lst = [str(x[0])+'='+str(x[1]) for x in zip(kwargs.keys(), kwargs.values())]
        query += '&'.join(lst) ## add all limits from kwargs
    uris = [] 
    #print(query)
    
    response = requests.get(query, 
               headers={"Content-Type":"application/json", 
                        "Authorization":f"Bearer {token}"})
    json_response = response.json()
    #print('Recommended Songs:')
    uris = []
    for i,j in enumerate(json_response['tracks']):
        uris.append(j['uri'])
        #print(f"{i+1}) \"{j['name']}\" by {j['artists'][0]['name']}")
    return uris

### Filter recommended songs

In [6]:
def filter_energy(r, pre_energy, market, trend = None):
    ''' filter recommended songs according to the energy trend '''
    
    if trend is None or pre_energy == 0:
        return r[0]
    
    energys_diff = [get_song(user_id, token, r[i][14:], market)['energy'][0] - pre_energy for i in range(len(r))]
    if trend == '+':
        if all(np.array(energys_diff) <= 0):
            return None
        idx = np.argmax( np.array(energys_diff) > 0)
    else:
        if all(np.array(energys_diff) >= 0):
            return None
        idx = np.argmax( np.array(energys_diff) < 0)
    return(r[idx])

### Create a playlist

In [7]:
# create a new playlist to store those recommendations
def create_plst(user_id, token, uris, name, description, public):
    endpoint_url = f"https://api.spotify.com/v1/users/{user_id}/playlists"
    request_body = json.dumps({
        "name": name,
        "description": description,
        "public": public })
    # create an empty new playlist
    response = requests.post(url = endpoint_url, data = request_body, headers={"Content-Type":"application/json", 
                        "Authorization":f"Bearer {token}"})
    url = response.json()['external_urls']['spotify']
    if response.status_code == 201:
        print('Playlist {} is successfully created!'.format(name))
        
    # fill the new playlist with the recommendations
    playlist_id = response.json()['id']
    endpoint_url = f"https://api.spotify.com/v1/playlists/{playlist_id}/tracks"
    request_body = json.dumps({ "uris" : uris })
    response = requests.post(url = endpoint_url, data = request_body, headers={"Content-Type":"application/json", 
                        "Authorization":f"Bearer {token}"})
    if response.status_code == 201:
        print('Playlist {} is successfully filled with recommendations!'.format(name))
        print(f'Your playlist is ready at {url}')

### generate a similar playlist

In [27]:
def get_recommendation_for_a_playlist(user_id, token, playlist_id, name, description, limit, time_percent, percents, public=False, **kwargs):
    print(playlist_id)
    market = kwargs.get("market", 'US')
    kwargs["market"] = market
    
    def _get_one_recommendation(idx, target_duration_ms, limit, **kwargs):
        df_response = get_song(user_id, token, idx, market)
        r = recommend(df_response, limit, time_percent, percents, target_duration_ms, **kwargs)

        #print(f"For the {j+1}-th song in the playlist, there are no recommendations based on your conditions, we modified the conditions.")
        new_time_percent = time_percent
        new_percent = percents
        i = 0
        while len(r) == 0 and i < 10:
            i += 1
            new_time_percent += 10
            new_percent *= 2
            r = recommend(df_response, limit, new_time_percent, new_percent, target_duration_ms, **kwargs)
        if i == 10:
            ## right now I keep the original one, we can improve it later
            r = ['spotify:track:' + df_response.index[0]]
        return r
        
    uris = []
    duration = 0
    ids, durations = get_ID(user_id, token, playlist_id, market)
    diff = 0
    pre_energy = ene = 0
    trend = None
    for j in range(len(ids)):
        energy = get_song(user_id, token, ids[j], market)['energy'][0] 
        trend = '+' if energy > pre_energy else '-'
        _limit = limit
        _r = None
        i = 0
        while _r is None and i < 10:
            ## if no songs satisfy requirements, we get more recommendations from API and then filter
            r = _get_one_recommendation(ids[j], target_duration_ms=durations[j] - diff, limit = _limit, **kwargs)    
            print(ene)
            print(trend)
            if len(r) < _limit:
                break
            _r = filter_energy(r, ene, market, trend) ## add limitations to filter the recommendations and keep the energy trend
            _limit = _limit + 10
            i = i+1
        if i == 10 or len(r) < _limit:
            _r = r[0]
            #_r = ['spotify:track:' + ids[j]]
        print(_r)
        uris.append(_r)
        dur = get_song(user_id, token, _r[14:], market)['duration_ms'][0] ## duration of the recommended song
        duration += dur
        diff = duration - sum(durations[:j+1])
        ene = get_song(user_id, token, _r[14:], market)['energy'][0] ## energy of the recommended song
        pre_energy = energy  ## energy of the j-th song in the playlist

    create_plst(user_id, token, [x for x in uris if x is not None], name, description, public)
    return([x for x in uris if x is not None])

## Test

Note: token should have scope at least "playlist-modify-private", you can find get one at the botton GET TOKEN from [here](https://developer.spotify.com/console/get-recommendations/?limit=10&market=ES&seed_artists=4NHQUGzhtTLFvgF5SZesLK&seed_genres=classical%2Ccountry&seed_tracks=0c6xIDDpzE81m2q797ordA&min_acousticness=&max_acousticness=&target_acousticness=&min_danceability=&max_danceability=&target_danceability=&min_duration_ms=&max_duration_ms=&target_duration_ms=&min_energy=&max_energy=&target_energy=&min_instrumentalness=&max_instrumentalness=&target_instrumentalness=&min_key=&max_key=&target_key=&min_liveness=&max_liveness=&target_liveness=&min_loudness=&max_loudness=&target_loudness=&min_mode=&max_mode=&target_mode=&min_popularity=&max_popularity=&target_popularity=&min_speechiness=&max_speechiness=&target_speechiness=&min_tempo=&max_tempo=&target_tempo=&min_time_signature=&max_time_signature=&target_time_signature=&min_valence=&max_valence=&target_valence=).

User id can be obtained from [your Spotify profile](https://www.spotify.com/us/account/overview/?utm_source=spotify&utm_medium=menu&utm_campaign=your_account).

In [13]:
# settings
token = "BQCBJzjT-1VYDujUlVJg0YsUmUaugcT2Z9kA8eEdSdtowXPFPh5FjqmXQuUFGyunVA5-leqb9bhGJIPg2FCJXelL-eQbJEH2i8NTNUEBtkwUZwvoOb-8qW8GNHNuw9uh522mXPLeaBK7zjono5gqOaWXzVvXxYaYDg6ZB8XiwMTcelYOK0E4eySf2BGLZTF04DZidQrP8BtxpA"
user_id = "pbwppse1hilahmk43ls424ao4"

name = 'previous'   ## name for the new playlist
description = '1101' ## description for the new playlist
limit = 10  ## for each song in current playlist, 10 recommendations are generated
time_percent = 15  ## the time interval width is 30s
percents = 0.1  ## the energy is current song's energy * (0.9, 1.1)

plst_id = get_plst_ID(user_id, token)  ## get id for each playlist from a user
#playlist_id = plst_id[6]    ## set the 2nd playlist to be the target playlist
playlist_id = '4N2UHn9HpFc3n93s1gduIM'

In [10]:
# run it
get_recommendation_for_a_playlist(user_id, token, playlist_id, name, description, limit, time_percent, percents, market='US')

4N2UHn9HpFc3n93s1gduIM
0
+
spotify:track:6n0E5CPhtGPLW1544OlSTg
0.562
+
spotify:track:00P5cJ8I9pNKkWXtHm6BDK
0.697
-
spotify:track:4VAdq9M22v99JHIXNv8TZ6
0.598
-
spotify:track:4Uw7NtaXY0xJrbR9qiaN4H
0.634
+
spotify:track:4GrcbD2aHVUvn0KvvnlKeR
0.755
+
spotify:track:5aoJnOhycrs0NtXomySi3e
0.819
+
spotify:track:4JunvrE0Vca2ETi0DtAc0W
0.842
-
spotify:track:0lc7EIQwNUFeInTzlL5ZW5
0.946
+
spotify:track:6sgwnGLqUj2Nw9UaPo7wNI
0.718
-
spotify:track:5V8QKyt40AbgQXIhGwhT5f
0.555
-
spotify:track:4MvbRbrOEsJgdYRGNGBjTE
0.454
-
spotify:track:41EZTXxSzbAjsOrQoY75FL
0.297
-
spotify:track:0AfIsqZ4gTUg9CwwW2jLeK
Playlist previous is successfully created!
Playlist previous is successfully filled with recommendations!
Your playlist is ready at https://open.spotify.com/playlist/5rHm7ksBQew2s9yaNv7Loy


### Measure of Similarity

In [16]:
market = 'US'
ids, durations = get_ID(user_id, token, playlist_id, market)

In [ ]:
pars = ['acousticness', 'danceability', 'energy', 'instrumentalness', 'key', 'liveness', 'loudness', 'mode', 
        'popularity', 'speechiness', 'tempo', 'time_signature', 'valence']

In [18]:
def weighted_metric(weight, par_name, playlist_id, ):
    ids, durations = get_ID(user_id, token, playlist_id, market)
    recommend_pl = get_recommendation_for_a_playlist(user_id, token, playlist_id, name, description, limit, time_percent, percents, market='US')
    for j in range(len(ids)):
        song = get_song(user_id, token, ids[j], market).iloc[0]
        original = song[par_name]


In [23]:
j = 0
song = get_song(user_id, token, ids[j], market).iloc[0]

In [ ]:
recommend_pl = get_recommendation_for_a_playlist(user_id, token, playlist_id, name, description, limit, time_percent, percents, market='US')

#### Check your new playlist in your Spotify!